In [ ]:
!apt install -y libspatialindex-dev
!pip install geopandas rasterio shapely pyproj folium osmnx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libspatialindex-dev is already the newest version (1.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
!pip install fiona

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 77.3 MB/s eta 0:00:00


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from rasterio.mask import mask
import matplotlib.pyplot as plt
from shapely.geometry import box, mapping, Point
import fiona
import os


In [ ]:
turkey = gpd.read_file("gadm41_TUR_0.shp").to_crs(epsg=3857)
turkey['geometry'] = turkey.geometry.simplify(1000)


minx, miny, maxx, maxy = turkey.total_bounds
cell_size = 1000
grid_cells = []
for x in np.arange(minx, maxx, cell_size):
    for y in np.arange(miny, maxy, cell_size):
        cell = box(x, y, x + cell_size, y + cell_size)
        grid_cells.append(cell)
grid = gpd.GeoDataFrame(geometry=grid_cells, crs=turkey.crs)


grid = grid[grid.geometry.centroid.within(turkey.unary_union)]


<ipython-input-3-779074ea082c>:16: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  grid = grid[grid.geometry.centroid.within(turkey.unary_union)]


In [ ]:
grid.to_file("turkey_grid_1km.gpkg", driver="GPKG")

In [ ]:
with rasterio.open('/content/drive/MyDrive/data/raw/ghi.tif') as src:
    grid['ghi'] = grid.geometry.centroid.apply(
        lambda p: list(src.sample([(p.x, p.y)]))[0][0]
    )


In [ ]:
raster_path = "GHI.tif"
with rasterio.open(raster_path) as src:

    grid_centroids = grid.geometry.centroid.to_crs(src.crs)
    ghi_values = []
    for point in grid_centroids:
        try:
            value = list(src.sample([(point.x, point.y)]))[0][0]
            ghi_values.append(value)
        except:
            ghi_values.append(np.nan)

grid["ghi_daily"] = ghi_values
grid["ghi_annual"] = grid["ghi_daily"] * 365.25


print(grid[["site_id", "ghi_daily", "ghi_annual"]].head())


grid.to_file("candidate_sites_with_ghi.geojson", driver="GeoJSON")
grid.drop(columns="geometry").to_parquet("candidate_sites_with_ghi.parquet")

KeyError: "['site_id'] not in index"

In [ ]:
grid["site_id"] = range(len(grid))

In [ ]:

print(grid[["site_id", "ghi_daily", "ghi_annual"]].head())


grid.to_file("candidate_sites_with_ghi.geojson", driver="GeoJSON")
grid.drop(columns="geometry").to_parquet("candidate_sites_with_ghi.parquet")

      site_id  ghi_daily   ghi_annual
608         0      4.554  1663.348464
609         1      4.538  1657.504539
610         2      4.465  1630.841306
611         3      4.488  1639.241969
1510        4      4.549  1661.522172


In [ ]:
grid.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1298185 entries, 608 to 1924168
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   geometry    1298185 non-null  geometry
 1   ghi_daily   1298185 non-null  float32 
 2   ghi_annual  1298185 non-null  float64 
 3   site_id     1298185 non-null  int64   
dtypes: float32(1), float64(1), geometry(1), int64(1)
memory usage: 44.6 MB


In [ ]:
import geopandas as gpd
import pandas as pd


corine = gpd.read_file("U2018_CLC2018_V2020_20u1.shp")
print("CORINE verisi yüklendi. Satır sayısı:", len(corine))


corine = corine.to_crs(grid.crs)


corine = corine[["Code_18", "geometry"]]
corine = corine.rename(columns={"CODE_18": "landuse_code"})


grid_landuse = gpd.sjoin(grid, corine, how="left", predicate="intersects")

unsuitable_codes = list(range(100, 400)) + list(range(500, 600))
grid_landuse["is_suitable_land"] = ~grid_landuse["landuse_code"].isin(unsuitable_codes)

corine_labels = {
    111: "Yapılaşmış alan",
    211: "Tarla",
    311: "Orman",
    512: "Su kütlesi",
    523: "Tuzlu bataklık"
}
grid_landuse["landuse_label"] = grid_landuse["landuse_code"].map(corine_labels)

grid_landuse.to_file("candidate_sites_with_corine.gpkg", driver="GPKG")


CORINE verisi yüklendi. Satır sayısı: 208507


KeyError: 'landuse_code'

In [ ]:
unsuitable_codes = list(range(100, 400)) + list(range(500, 600))
grid_landuse["is_suitable_land"] = ~grid_landuse["Code_18"].isin(unsuitable_codes)


corine_labels = {
    111: "Yapılaşmış alan",
    211: "Tarla",
    311: "Orman",
    512: "Su kütlesi",
    523: "Tuzlu bataklık"
}
grid_landuse["landuse_label"] = grid_landuse["Code_18"].map(corine_labels)


grid_landuse.to_file("candidate_sites_with_corine.gpkg", driver="GPKG")

In [ ]:
plausible_codes = [
    321, 322, 323, 324,       # Çalılık ve geçişli alanlar
    331, 332, 333, 334, 335,  # Doğal bozkır ve taşlık
    411, 421, 422             # Doğal yarı açıklık alanlar
]


grid_landuse = grid_landuse[grid_landuse["Code_18"].notna()]
grid_landuse["Code_18"] = grid_landuse["Code_18"].astype(int)


grid_landuse["is_suitable_land"] = grid_landuse["Code_18"].isin(plausible_codes)


/usr/local/lib/python3.11/dist-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [ ]:
unsuitable_codes = list(range(100, 400)) + list(range(500, 600))


grid_landuse.loc[grid_landuse["Code_18"].notna(), "Code_18"] = (
    grid_landuse.loc[grid_landuse["Code_18"].notna(), "Code_18"].astype(int)
)


grid_landuse["is_suitable_land"] = ~grid_landuse["Code_18"].isin(unsuitable_codes)


In [ ]:

suitable_counts = grid_landuse.groupby("site_id")["is_suitable_land"].sum()


valid_site_ids = suitable_counts[suitable_counts >= 2].index


grid_landuse_filtered = grid_landuse[grid_landuse["site_id"].isin(valid_site_ids)].copy()


In [ ]:
print(f"Toplam filtrelenmiş hücre sayısı: {len(grid_landuse_filtered)}")
print(f"Farklı site_id sayısı: {grid_landuse_filtered['site_id'].nunique()}")


Toplam filtrelenmiş hücre sayısı: 945397
Farklı site_id sayısı: 294416


In [ ]:
grid_landuse_filtered = grid_landuse_filtered.drop_duplicates(subset="site_id", keep="first")


In [ ]:
grid_landuse = grid_landuse.drop(columns="landuse_label")

In [ ]:
print("Uygun olmayan hücre sayısı:", (grid_landuse["is_suitable_land"] == False).sum())


Uygun olmayan hücre sayısı: 1950564


In [ ]:
print("Uygun olmayan hücre sayısı:", (grid_landuse["is_suitable_land"] == False).sum())


Uygun olmayan hücre sayısı: 3090024


In [ ]:
print("Uygun olan hücre sayısı:", (grid_landuse["is_suitable_land"] == True).sum())

Uygun olan hücre sayısı: 15328


In [ ]:
print("Uygun olan hücre sayısı:", (grid_landuse["is_suitable_land"] == True).sum())

Uygun olan hücre sayısı: 1153500


In [ ]:
grid_landuse["Code_18"].value_counts().sort_index()


Code_18
111      5165
112     59775
121     13978
122      5200
123       266
124      1237
131     10267
132       418
133      6024
141       935
142      2541
211    355674
212    188283
213      5440
221      9583
222     50496
223     16058
231    102957
241       132
242    213037
243    371738
244        77
311    149525
312    215275
313    115721
321    332532
322        13
323     36918
324    363412
331      7555
332     65926
333    332928
334       151
335        25
411      8849
421      4724
422       467
511     10709
512     34647
521       463
522         9
523      4934
Name: count, dtype: int64

In [ ]:
print(corine.columns)

Index(['OBJECTID', 'Code_18', 'Remark', 'Area_Ha', 'ID', 'Shape_Leng',
       'Shape_Area', 'geometry'],
      dtype='object')


In [ ]:
print(grid_landuse.columns)

Index(['geometry', 'ghi_daily', 'ghi_annual', 'site_id', 'index_right',
       'Code_18'],
      dtype='object')


In [ ]:
suitable_only = grid_landuse[grid_landuse["is_suitable_land"] == True].copy()


In [ ]:
true_counts = suitable_only.groupby("site_id").size()
valid_site_ids = true_counts[true_counts >= 2].index


In [ ]:

filtered = (
    suitable_only[suitable_only["site_id"].isin(valid_site_ids)]
    .sort_values(["site_id", "Code_18"], ascending=[True, False])
    .drop_duplicates(subset="site_id", keep="first")
    .copy()
)


In [ ]:
print("Kalan site_id sayısı:", filtered["site_id"].nunique())
print("Toplam satır:", len(filtered))
filtered.head()


Kalan site_id sayısı: 294416
Toplam satır: 294416


geometry  ghi_daily  \
608   POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
609   POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
610   POLYGON ((2858184.906 4885450.791, 2858184.906...      4.465   
611   POLYGON ((2858184.906 4886450.791, 2858184.906...      4.488   
1510  POLYGON ((2859184.906 4883450.791, 2859184.906...      4.549   

       ghi_annual  site_id  index_right  Code_18  is_suitable_land  
608   1663.348464        0      57972.0      331              True  
609   1657.504539        1      56414.0      323              True  
610   1630.841306        2      56519.0      324              True  
611   1639.241969        3      57999.0      333              True  
1510  1661.522172        4      57972.0      331              True

In [ ]:
print("Uygun olmayan hücre sayısı:", (filtered["is_suitable_land"] == False).sum())

Uygun olmayan hücre sayısı: 0


In [ ]:
grid_landuse.head(10)

geometry  ghi_daily  \
608  POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
608  POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
608  POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
608  POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
609  POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
609  POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
609  POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
609  POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
610  POLYGON ((2858184.906 4885450.791, 2858184.906...      4.465   
610  POLYGON ((2858184.906 4885450.791, 2858184.906...      4.465   

      ghi_annual  site_id  index_right  Code_18  is_suitable_land  
608  1663.348464        0     207753.0      523             False  
608  1663.348464        0      57972.0      331              True  
608  1663.348464        0      56414.0      323              True  
608  1663.348464        0      55069.0      312             False  
609  1657.504539        1     207753.0      523             False  
609  1657.504539        1      56414.0      323              True  
609  1657.504539        1      55069.0      312             False  
609  1657.504539        1      55955.0      321              True  
610  1630.841306        2     207753.0      523             False  
610  1630.841306        2      55955.0      321              True

In [ ]:
unsuitable = grid_landuse[grid_landuse["is_suitable_land"] == False]

In [ ]:
unsuitable.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1950564 entries, 608 to 1922702
Data columns (total 8 columns):
 #   Column            Dtype   
---  ------            -----   
 0   geometry          geometry
 1   ghi_daily         float32 
 2   ghi_annual        float64 
 3   site_id           int64   
 4   index_right       float64 
 5   Code_18           int64   
 6   is_suitable_land  bool    
 7   landuse_label     object  
dtypes: bool(1), float32(1), float64(2), geometry(1), int64(2), object(1)
memory usage: 113.5+ MB


In [ ]:
grid_landuse.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3104064 entries, 608 to 1922702
Data columns (total 8 columns):
 #   Column            Dtype   
---  ------            -----   
 0   geometry          geometry
 1   ghi_daily         float32 
 2   ghi_annual        float64 
 3   site_id           int64   
 4   index_right       float64 
 5   Code_18           int64   
 6   is_suitable_land  bool    
 7   landuse_label     object  
dtypes: bool(1), float32(1), float64(2), geometry(1), int64(2), object(1)
memory usage: 180.6+ MB


In [ ]:
grid_landuse.to_file("grid_landuse_final.gpkg", driver="GPKG")


In [ ]:
I = filtered[
    (filtered["is_suitable_land"] == True) &
    (filtered["ghi_annual"] >= 1600)
].copy()


In [ ]:
print("Toplam uygun kurulum alanı sayısı:", len(I))
I.head()


Toplam uygun kurulum alanı sayısı: 231799


geometry  ghi_daily  \
608   POLYGON ((2858184.906 4883450.791, 2858184.906...      4.554   
609   POLYGON ((2858184.906 4884450.791, 2858184.906...      4.538   
610   POLYGON ((2858184.906 4885450.791, 2858184.906...      4.465   
611   POLYGON ((2858184.906 4886450.791, 2858184.906...      4.488   
1510  POLYGON ((2859184.906 4883450.791, 2859184.906...      4.549   

       ghi_annual  site_id  index_right  Code_18  is_suitable_land  
608   1663.348464        0      57972.0      331              True  
609   1657.504539        1      56414.0      323              True  
610   1630.841306        2      56519.0      324              True  
611   1639.241969        3      57999.0      333              True  
1510  1661.522172        4      57972.0      331              True

In [ ]:
I.to_file("grid_final_1600_I.gpkg", driver="GPKG")


In [ ]:
import geopandas as gpd

osm_substations = gpd.read_file("export.geojson")
osm_substations = osm_substations[osm_substations.geometry.type == "Point"]
osm_substations = osm_substations.to_crs("EPSG:4326")


In [ ]:
provinces = gpd.read_file("gadm41_TUR_1.shp")
provinces = provinces.to_crs("EPSG:4326")
provinces["NAME_1"] = provinces["NAME_1"].str.title()


In [ ]:

osm_with_province = gpd.sjoin(osm_substations, provinces[["NAME_1", "geometry"]], how="left", predicate="within")
osm_with_province = osm_with_province.rename(columns={"NAME_1": "province"})

In [ ]:

osm_with_province = gpd.sjoin(osm_substations, provinces[["NAME_1", "geometry"]], how="left", predicate="within")
osm_with_province = osm_with_province.rename(columns={"NAME_1": "province"})


osm_counts = osm_with_province["province"].value_counts().to_dict()


In [ ]:
teias_trafo_counts = {
    "Adana": 24,
    "Adıyaman": 8,
    "Afyonkarahisar": 8,
    "Ağrı": 5,
    "Aksaray": 5,
    "Amasya": 4,
    "Ankara": 35,
    "Antalya": 26,
    "Ardahan": 2,
    "Artvin": 9,
    "Aydın": 11,
    "Balıkesir": 13,
    "Bartın": 2,
    "Batman": 4,
    "Bayburt": 1,
    "Bilecik": 7,
    "Bingöl": 4,
    "Bitlis": 4,
    "Bolu": 5,
    "Burdur": 4,
    "Bursa": 24,
    "Çanakkale": 10,
    "Çankırı": 5,
    "Çorum": 6,
    "Denizli": 12,
    "Diyarbakır": 13,
    "Düzce": 5,
    "Edirne": 7,
    "Elazığ": 9,
    "Erzincan": 5,
    "Erzurum": 11,
    "Eskişehir": 9,
    "Gaziantep": 14,
    "Giresun": 9,
    "Gümüşhane": 3,
    "Hakkari": 3,
    "Hatay": 11,
    "Iğdır": 2,
    "Isparta": 7,
    "İstanbul": 62,
    "İzmir": 48,
    "Kahramanmaraş": 16,
    "Karabük": 4,
    "Karaman": 6,
    "Kars": 3,
    "Kastamonu": 9,
    "Kayseri": 13,
    "Kırıkkale": 2,
    "Kırklareli": 8,
    "Kırşehir": 4,
    "Kilis": 2,
    "Kocaeli": 14,
    "Konya": 29,
    "Kütahya": 8,
    "Malatya": 7,
    "Manisa": 14,
    "Mardin": 10,
    "Mersin": 19,
    "Muğla": 13,
    "Muş": 2,
    "Nevşehir": 5,
    "Niğde": 5,
    "Ordu": 6,
    "Osmaniye": 6,
    "Rize": 6,
    "Sakarya": 9,
    "Samsun": 13,
    "Siirt": 4,
    "Sinop": 3,
    "Sivas": 11,
    "Şanlıurfa": 26,
    "Şırnak": 5,
    "Tekirdağ": 13,
    "Tokat": 8,
    "Trabzon": 9,
    "Tunceli": 3,
    "Uşak": 6,
    "Van": 6,
    "Yalova": 1,
    "Yozgat": 7,
    "Zonguldak": 7
}


In [ ]:
needed_synthetics = {}

for city, required in teias_trafo_counts.items():
    current = osm_counts.get(city, 0)
    if current < required:
        needed_synthetics[city] = required - current


In [ ]:
import random
from shapely.geometry import Point

In [ ]:
synthetic_points = []
synthetic_labels = []

for city, count in needed_synthetics.items():
    geom = provinces[provinces["NAME_1"] == city].unary_union
    minx, miny, maxx, maxy = geom.bounds

    added = 0
    tries = 0
    while added < count and tries < 1000:
        x = random.uniform(minx, maxx)
        y = random.uniform(miny, maxy)
        point = Point(x, y)
        if geom.contains(point):
            synthetic_points.append(point)
            synthetic_labels.append(city)
            added += 1
        tries += 1


<ipython-input-13-89310e85dccf>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = provinces[provinces["NAME_1"] == city].unary_union
<ipython-input-13-89310e85dccf>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = provinces[provinces["NAME_1"] == city].unary_union
<ipython-input-13-89310e85dccf>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = provinces[provinces["NAME_1"] == city].unary_union
<ipython-input-13-89310e85dccf>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = provinces[provinces["NAME_1"] == city].unary_union
<ipython-input-13-89310e85dccf>:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = provinces[provinces["NAME_1"] == city].unary_union
<ipython-input-13-89

In [ ]:
import pandas as pd

In [ ]:
synthetic_gdf = gpd.GeoDataFrame({
    "source": "synthetic",
    "province": synthetic_labels,
    "geometry": synthetic_points
}, crs="EPSG:4326")

final_substations = pd.concat([osm_with_province, synthetic_gdf], ignore_index=True)


In [ ]:
final_substations.sample(20)

id               @id addr:city addr:country  \
420               NaN               NaN       NaN          NaN   
328  node/12417229129  node/12417229129      None         None   
340  node/12543662018  node/12543662018      None         None   
222  node/11112894433  node/11112894433      None         None   
685               NaN               NaN       NaN          NaN   
432               NaN               NaN       NaN          NaN   
366  node/12858104261  node/12858104261      None         None   
500               NaN               NaN       NaN          NaN   
548               NaN               NaN       NaN          NaN   
557               NaN               NaN       NaN          NaN   
568               NaN               NaN       NaN          NaN   
78    node/5840048961   node/5840048961      None         None   
238  node/11281785345  node/11281785345      None         None   
305  node/12145205396  node/12145205396      None         None   
382               NaN               NaN       NaN          NaN   
250  node/11545977449  node/11545977449      None         None   
507               NaN               NaN       NaN          NaN   
71    node/5156320209   node/5156320209      None         None   
111   node/6856031137   node/6856031137      None         None   
771               NaN               NaN       NaN          NaN   

    addr:housenumber addr:postcode addr:province addr:street alt_name  \
420              NaN           NaN           NaN         NaN      NaN   
328             None          None          None        None     None   
340             None          None          None        None     None   
222             None          None          None        None     None   
685              NaN           NaN           NaN         NaN      NaN   
432              NaN           NaN           NaN         NaN      NaN   
366             None          None          None        None     None   
500              NaN           NaN           NaN         NaN      NaN   
548              NaN           NaN           NaN         NaN      NaN   
557              NaN           NaN           NaN         NaN      NaN   
568              NaN           NaN           NaN         NaN      NaN   
78              None          None          None        None     None   
238             None          None          None        None     None   
305             None          None          None        None     None   
382              NaN           NaN           NaN         NaN      NaN   
250             None          None          None        None     None   
507              NaN           NaN           NaN         NaN      NaN   
71              None          None          None        None     None   
111             None          None          None        None     None   
771              NaN           NaN           NaN         NaN      NaN   

    building  ...       power rating   ref     source          substation  \
420      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
328     None  ...  substation   None  None       None                None   
340     None  ...  substation   None  None       None  minor_distribution   
222     None  ...  substation   None  None       None                None   
685      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
432      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
366     None  ...  substation   None  None       None                None   
500      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
548      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
557      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
568      NaN  ...         NaN    NaN   NaN  synthetic                 NaN   
78      None  ...  substation   None  None       None                None   
238     None  ...  substation   None  None       None                None   
305     None  ...  s

In [ ]:
final_substations.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 795 entries, 0 to 794
Data columns (total 29 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 367 non-null    object  
 1   @id                367 non-null    object  
 2   addr:city          3 non-null      object  
 3   addr:country       4 non-null      object  
 4   addr:housenumber   2 non-null      object  
 5   addr:postcode      2 non-null      object  
 6   addr:province      2 non-null      object  
 7   addr:street        1 non-null      object  
 8   alt_name           1 non-null      object  
 9   building           11 non-null     object  
 10  fixme              2 non-null      object  
 11  frequency          58 non-null     object  
 12  gas_insulated      1 non-null      object  
 13  location           51 non-null     object  
 14  man_made           1 non-null      object  
 15  name               70 non-null     object  
 16  

In [ ]:
final_substations = gpd.GeoDataFrame(final_substations, geometry="geometry", crs="EPSG:4326")

In [ ]:
final_substations.to_file("final_substations.geojson", driver="GeoJSON")